In [9]:
#WARNING THIS SCRIPT TAKES A LONG TIME TO RUN!
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
from CMS_SURF_2016.utils.data_parse import ROOT_to_pandas
from CMS_SURF_2016.utils.data_parse import DataProcessingProcedure
from CMS_SURF_2016.utils.data_parse import leaves_from_obj
import ROOT
from ROOT import TTree
import numpy as np
import pandas as pd
import ntpath
import glob
from itertools import cycle, islice



In [10]:
def cullNonObservables(frame):
    #Status of 1 means that the particle is a stable product
    stable_cond = frame["Status"] == 1 
    #All even leptons are neutrinos which we can't measure
    notNeutrino_cond = frame["PID"] % 2 == 1
    parton_cond = np.abs(frame["PID"]) <= 8
    #Get all entries that satisfy the conditions
    frame = frame[stable_cond & notNeutrino_cond]
    #Drop the Status frame since we only needed it to see if the particle was stable
    frame = frame.drop(["Status"], axis=1)
    return frame
#Define the speed of light C
C = np.float64(2.99792458e8);  
def four_vec_func(inputs):
        E = inputs[0]
        Eta = inputs[1]
        Phi = inputs[2]
        E_over_c = E/C
        px = E_over_c * np.sin(Phi) * np.cos(Eta) 
        py = E_over_c * np.sin(Phi) * np.sin(Eta)
        pz = E_over_c * np.cos(Phi)
        return [E_over_c, px, py, pz]
    
def getPandasPhotons(filename):
    four_vec_inputs, dummy = leaves_from_obj("Photon", ["E", "Eta", "Phi"])
    four_vec_proc = DataProcessingProcedure(four_vec_func, four_vec_inputs, ["E/c", "Px","Py","Pz"])

    PID_proc = DataProcessingProcedure(lambda x:[22], [], ["PID"])
    charge_proc = DataProcessingProcedure(lambda x:[0], [], ["Charge"])

    columns=[four_vec_proc, PID_proc, charge_proc]
    leaves, columns = leaves_from_obj("Photon", columns)

    #Extract the tables from the root file
    photon_frame = ROOT_to_pandas(filename,
                          leaves,
                          columns=columns,
                          verbosity=1)
    return photon_frame

def getPandasParticles(filename, cull=True):
    E_over_c_proc = DataProcessingProcedure(lambda x:x[0]/C, ["Particle.E"], ["E/c"])
    columns= [E_over_c_proc, "Px", "Py", "Pz", "PID", "Charge", "Status"]
    leaves, columns = leaves_from_obj("Particle", columns)
    original_frame = ROOT_to_pandas(filename,
                                 leaves,
                                  columns=columns,
                                  verbosity=1)
    if(cull):
        particle_frame = cullNonObservables(original_frame)
    else:
        particle_frame = original_frame
    return particle_frame

def getPandasObservables(filename):
    out = pd.concat([getPandasPhotons(filename),getPandasParticles(filename, cull=True)])
    return out

def getPandasAll(filename):
    out = pd.concat([getPandasPhotons(filename),getPandasParticles(filename, cull=False)])
    return out

#http://stackoverflow.com/questions/3678869/pythonic-way-to-combine-two-lists-in-an-alternating-fashion
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    pending = len(iterables)
    nexts = cycle(iter(it).next for it in iterables)
    while pending:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            pending -= 1
            nexts = cycle(islice(nexts, pending))
    
#def groupEntriesToArrays(frame, select):
#    out = []
#    m = frame['Entry'].max()
#    for entry in range(0, m+1):
#        cond = frame['Entry'] == entry
#        entry_frame = frame[cond]
#        entry_frame = entry_frame[select]
#        arr = np.array(entry_frame)
#        np.random.shuffle(arr)
#        out.append(arr)
#    return out

#def groupEntriesToArrays(frame, select):
#    grouped = frame.groupby(["Entry"]).apply(lambda df: df[select].tolist())
    #print(grouped)
#    return grouped

    

In [ ]:
ttbar_files = glob.glob("/data/shared/Delphes/ttbar_13TeV/*.root")
WJet_files = glob.glob("/data/shared/Delphes/WJets_lepFilter_13TeV/*.root")



ttbar_out_dir = "/data/shared/Delphes/ttbar_13TeV/parsed_pandas/"
if not os.path.exists(ttbar_out_dir):
    os.makedirs(ttbar_out_dir)
    
WJet_out_dir = "/data/shared/Delphes/WJets_13TeV/parsed_pandas/"
if not os.path.exists(WJet_out_dir):
    os.makedirs(WJet_out_dir)

ttbar_jobs = []
WJet_jobs = []
    
for ttbar_file in ttbar_files:
    ttbar_filename = os.path.splitext(ntpath.basename(ttbar_file))[0]
    ttbar_jobs.append((ttbar_file, ttbar_out_dir + ttbar_filename + ".h5"))

for WJet_file in WJet_files
    WJet_filename = os.path.splitext(ntpath.basename(WJet_file))[0]
    WJet_jobs.append((WJet_file, WJet_out_dir + WJet_filename + ".h5"))
    
jobs = roundrobin(ttbar_jobs, WJet_jobs)

In [ ]:
for job in jobs:
    #print(0,job[0])
    #print(1,job[1])
    frame = getPandasAll(job[0])
    frame.to_hdf(job[1], 'data', mode='w')

Extracting data from /data/shared/Delphes/WJets_lepFilter_13TeV/WJets_lepFilter_13TeV_0.root
Using trees: Delphes
Procedure at column 1 maps "['Photon.E', 'Photon.Eta', 'Photon.Phi'] -> ['E/c', 'Px', 'Py', 'Pz']"
Procedure at column 5 maps "[] -> ['PID']"
Procedure at column 6 maps "[] -> ['Charge']"
Extracting Leaves: Photon.E, Photon.Eta, Photon.Phi
Renaming to: E/c, Px, Py, Pz, PID, Charge
[=================== ] 9647/10000      Entry           E/c            Px            Py            Pz  PID  \
0         6  1.497562e-06  8.703046e-07 -4.698388e-07 -1.124505e-06   22   
1         9  5.229001e-07 -7.278442e-09 -2.979777e-09 -5.228409e-07   22   
2         9  3.655351e-08 -1.951612e-08 -3.856315e-09  3.066609e-08   22   
3        13  1.496809e-07 -4.281884e-08  8.502223e-08 -1.155082e-07   22   
4        17  1.409828e-07  2.625648e-08  4.708176e-08  1.302692e-07   22   
...     ...           ...           ...           ...           ...  ...   
1701   9964  2.616170e-07  1.190883e-08